<a href="https://colab.research.google.com/github/marcelobenedito/quarantine_covid19_behavior_analysis/blob/master/quarantine_covid19_behavior_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Quarantine Covid-19 Behavior Analysis**

*It will be collect data tweets about COVID-19, quarantine and related about. This content will analysed to extract sentiment and main user behavior that makes don't stay home.*

**Install libraries**

In [3]:
!pip3 install unidecode
!pip3 install googletrans
!pip3 install twitterscraper
!pip3 install emoji

  Using cached https://files.pythonhosted.org/packages/71/3a/3b19effdd4c03958b90f40fe01c93de6d5280e03843cc5adf6956bfc9512/googletrans-3.0.0.tar.gz
     |████████████████████████████████| 61kB 1.9MB/s 
     |████████████████████████████████| 51kB 5.4MB/s 
     |████████████████████████████████| 931kB 8.4MB/s 
     |████████████████████████████████| 61kB 8.3MB/s 
     |████████████████████████████████| 71kB 9.3MB/s 
     |████████████████████████████████| 102kB 9.0MB/s 
  Created wheel for googletrans: filename=googletrans-3.0.0-cp36-none-any.whl size=15736 sha256=0a6122011f682194aaceecb62e93e52f37a48cfb72dc54b095cb91ae9aedd5ae
  Stored in directory: /root/.cache/pip/wheels/28/1a/a7/eaf4d7a3417a0c65796c547cff4deb6d79c7d14c2abd29273e
  Created wheel for contextvars: filename=contextvars-2.4-cp36-none-any.whl size=7666 sha256=19f144b6c60ebf386484311b10cd61c14836f7a6a71a0db36027f8ce7a624ee9
  Stored in directory: /root/.cache/pip/wheels/a5/7d/68/1ebae2668bda2228686e3c1cf16f2c2384cea6e9334ad

**Required imports**

In [19]:
import string
import time
import datetime as dt
import numpy as np
import pandas as pd
import re
import nltk
from unidecode import unidecode
from googletrans import Translator
from twitterscraper import query_tweets
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from emoji import demojize

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

**Create a funcion to search tweets**

In [ ]:
def search_tweets(search_filter, since, until, limit, language):
  return query_tweets(query = search_filter, begindate = since, enddate = until, limit = limit, lang = language)

**Defining filters used in search**

In [ ]:
""" não estou saindo "não estou saindo" (quarentena OR covid) (#covid-19 OR #coronavírus OR #coronavirus OR #covid OR #quarentena) lang:pt until:2020-01-31 since:2020-01-01 -filter:replies """

contains_both_words = ''
exact_phrase = ''
contains_any_words = '(quarentena OR covid OR coronavirus OR isolamento OR social)'
contains_any_hashtags = ''
no_retweet = '-filter:replies'
language = 'pt'
since = dt.date(2020,1,1)
until = dt.date(2020,1,31)
limit = 10

search_filter = contains_both_words + ' ' + exact_phrase + ' ' + contains_any_words + ' ' + contains_any_hashtags + ' ' + no_retweet

**Extracting tweets based on search filter**

In [ ]:
tweets = search_tweets(search_filter, since, until, limit, language)

**Transform Json to DataFrame and export to CSV file**

In [ ]:
df = pd.DataFrame({
    'tweet_id': tweet.tweet_id, 
    'text': tweet.text,  
    'tweet_url': tweet.tweet_url,
    'retweets': tweet.retweets,
    'replies': tweet.replies,
    'is_replied': tweet.is_replied,
    'is_reply_to': tweet.is_reply_to,
    'user_id': tweet.user_id, 
    'created_at': tweet.timestamp
} for tweet in tweets)

df.to_csv('tweets.csv', encoding='utf-8')

**Printing found tweets**

In [ ]:
df.head()

,tweet_id,text,tweet_url,retweets,replies,is_replied,is_reply_to,user_id,created_at
0,1213248633954611200,Danilo Gentili e deputado do Psol discutem em ...,/aratuonline/status/1213248633954611200,1,1,True,False,67661509,2020-01-03 23:59:50
1,1213248562420703232,"Jamais rebaixe alguém da família, pq essa pess...",/milamarques/status/1213248562420703232,0,0,False,False,256680355,2020-01-03 23:59:33
2,1213248430342115328,Já planejam diminuir conteúdo dos livros didát...,/rakavazquez/status/1213248430342115328,3,0,False,False,732761681856765952,2020-01-03 23:59:01
3,1213248424134500353,QUESTIONÁRIO PARA 2020\n \nEU VOU?\n1. Tomara ...,/gahsep1914/status/1213248424134500353,0,0,False,False,710262555223134208,2020-01-03 23:59:00
4,1213248395512631296,Só os especialistas em política e economia onl...,/NerdZEEH/status/1213248395512631296,0,0,False,False,281665822,2020-01-03 23:58:53


**Open stored tweets from CSV file**

In [ ]:
# file = open('tweets.csv', encoding='utf-8').read()
df = pd.read_csv('tweets.csv')


**Probably will need to make a translate from Portuguese to English**

In [ ]:
tweets = df['text']
for tweet in tweets:
  english_tweets = Translator().translate(unidecode(tweet)).text

**Data preprocessing**

This process is used to preprocess the tweet text:

 - Tokenize words;
 - Remove all stop words; 
 - Punctuaction rules; 
 - Unused characters;
 - Links from tweets.

In [14]:
tweets = ['The second-quarter contraction set a grim record, and it would have been worse without government aid that is expiring.', 
          'The coronavirus pandemic’s toll on the nation’s economy became emphatically clearer Thursday as the government detailed the most devastating three-month collapse on record, which wiped away nearly five years of growth.']
tweets = pd.DataFrame(tweets, columns=['text'])
tweets = tweets.text
tweets

0    The second-quarter contraction set a grim reco...
1    The coronavirus pandemic’s toll on the nation’...
Name: text, dtype: object

In [18]:
# Converting to lowercase
tweets = tweets.str.lower()

# Removing punctuation rules
tweets = tweets.str.translate(str.maketrans('', '', string.punctuation))

# Removing unused links
tweets = tweets.str.replace(r"(http|@)\S+", "")

# Remove special chars
tweets = tweets.apply(demojize)
tweets = tweets.str.replace(r"::", ": :")
tweets = tweets.str.replace(r"’", "'")
tweets = tweets.str.replace(r"[^a-z\':_]", " ")

# Remove repetitions
pattern = re.compile(r"(.)\1{2,}", re.DOTALL)
tweets = tweets.str.replace(pattern, r"\1")

# Transform short negation form
tweets = tweets.str.replace(r"(can't|cannot)", 'can not')
tweets = tweets.str.replace(r"n't", ' not')

# Spliting text into words
# tweets = word_tokenize(tweets, 'english')

# Removing stop words
stopwords = stopwords.words('english')
stopwords.remove('not')
stopwords.remove('nor')
stopwords.remove('no')

tweets.apply(
    lambda tweet: ' '.join([word for word in tweet.split() if word not in stopwords])
)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


0    secondquarter contraction set grim record woul...
1    coronavirus pandemics toll nations economy bec...
Name: text, dtype: object